In [50]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import words, stopwords
import keras
import pandas as pd
import numpy as np

from keras.preprocessing import text, sequence
import gensim.models.word2vec as w2v
import copy
from tqdm import tqdm
import multiprocessing as mp

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import maketrans 

In [94]:
train = pd.read_csv("./train_cleaned.csv")
test = pd.read_csv("./test_cleaned.csv")

In [91]:
def text_to_wordlist(text):
    
    # Convert words to lower case and split them
    text = text.lower()

    # Clean the text
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"'m", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return(text)

In [87]:
train = pd.read_csv("/Users/fgoebels/workspace/kaggle-toxicity/train.csv")
test = pd.read_csv("/Users/fgoebels/workspace/kaggle-toxicity/test.csv")

In [78]:
print(train.shape)
print(test.shape)

(95851, 8)
(226998, 2)


In [99]:
all_text = test["comment_text"].dropna().values
all_text = list(np.append(all_text, train["comment_text"].dropna().values))
all_text_clean = [""]*len(all_text)
for i in tqdm(range(len(all_text))):
    all_text_clean[i] = text_to_wordlist(all_text[i])
all_text_tokenized = [word_tokenize(s) for s in all_text_clean]
#print len(all_text)
#print len(all_text_tokenized)

100%|██████████| 318321/318321 [00:23<00:00, 13303.27it/s]


In [100]:
# hyper parameters of the word2vec model
num_features = 300 # dimensions of each word embedding
min_word_count = 1 # this is not advisable but since we need to extract
# feature vector for each word we need to do this
num_workers = mp.cpu_count() # number of threads running in parallel
context_size = 7 # context window length
downsampling = 1e-3 # downsampling for very frequent words
seed = 1 # seed for random number generator to make results reproducible

In [101]:
word2vec_ = w2v.Word2Vec(
    sg = 1, seed = seed,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context_size,
    sample = downsampling
)

In [102]:
# first we need to built the vocab
word2vec_.build_vocab(all_text_tokenized)

In [103]:
word2vec_.train(all_text_tokenized, total_examples = word2vec_.corpus_count, epochs = word2vec_.iter)

72449580

In [104]:
word2vec_.wv.most_similar('male')

[('female', 0.7587818503379822),
 ('females', 0.602188229560852),
 ('heterosexual', 0.60048907995224),
 ('genitalia', 0.5827293395996094),
 ('senility', 0.5809351205825806),
 ('males', 0.5778359174728394),
 ('femininity', 0.5775822401046753),
 ('transsexual', 0.5729291439056396),
 ('nohibokoma', 0.5721511244773865),
 ('nohiboka', 0.570717453956604)]

In [110]:
word2vec_.wv.most_similar('lolly')

[('metalpunkhardcore', 0.9332546591758728),
 ('luved', 0.9314939975738525),
 ('homofobe', 0.9314460754394531),
 ('flexin', 0.9312250018119812),
 ('nazorean', 0.9306169748306274),
 ('wahahhahaahah', 0.9297462701797485),
 ('giddily', 0.9291085600852966),
 ('underoos', 0.9285200834274292),
 ('fnidoasnfdklasnfeiowanfdsznvjdsafneiawnmdxsancjkdsafndsakjfeniwaofds',
  0.9280965924263),
 ('saidit', 0.9279337525367737)]

In [106]:
word2vec_.wv.most_similar('dick')

[('ullmann', 0.6498888731002808),
 ('cline', 0.6433099508285522),
 ('suck', 0.6330228447914124),
 ('syndrom', 0.6034367680549622),
 ('ballsack', 0.5976033210754395),
 ('whore', 0.5921552181243896),
 ('dickheadfuck', 0.5883691906929016),
 ('douchebag', 0.5871796011924744),
 ('faggoty', 0.5869563221931458),
 ('sucking', 0.5817825794219971)]

In [32]:
word2vec_.wv.most_similar('male')

[('female', 0.8036949634552002),
 ('females', 0.6898893117904663),
 ('males', 0.6633477807044983),
 ('heterosexual', 0.6574035882949829),
 ('femininity', 0.6503486633300781),
 ('senility', 0.6500623226165771),
 ('genitalia', 0.647432804107666),
 ('sexually', 0.6322382092475891),
 ('hetero', 0.6257021427154541),
 ('woman', 0.625482439994812)]

In [33]:
word2vec_.wv.most_similar('gay')

[('lolooolbootstoots', 0.6720337867736816),
 ('friggen', 0.6649272441864014),
 ('bunksteve', 0.6477774977684021),
 ('dashiel', 0.6474275588989258),
 ('cody', 0.6383813619613647),
 ('cline', 0.6378067135810852),
 ('homosexual', 0.6349483728408813),
 ('cyphoidbomb', 0.6246486902236938),
 ('loll', 0.622316300868988),
 ('lesbian', 0.6129813194274902)]

In [34]:
word2vec_.wv.most_similar('dick')

[('suck', 0.7001449465751648),
 ('ullmann', 0.66269850730896),
 ('cline', 0.6521153450012207),
 ('cock', 0.6502227783203125),
 ('ballsack', 0.6378535032272339),
 ('faggoty', 0.6336411833763123),
 ('whore', 0.631374180316925),
 ('laurent', 0.6251245141029358),
 ('sucking', 0.6240524053573608),
 ('chode', 0.6206081509590149)]